In [28]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1,1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [29]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 가중치변수 W는 [입력층(특징 수), 출력층(레이블 수)]의 구성인 [2, 3]으로 설정하고,
# 편향변수 b는 레이블 수인 3개의 요소를 가진 변수로 설정합니다.
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

In [30]:
# softmax 함수는 배열 내의 결과값들을 전체의 합이 1이 되도록 만들어 줍니다.
# 전체가 1이니 각각은 해당 결과의 확률로 해석할 수 있습니다.
model = tf.nn.softmax(L)

In [31]:
# Cost Function
# 원-핫 인코딩을 이용한 대부분의 모델에서 사용하는 교차 엔트로피 (Cross-Entropy)함수를 사용하겠습니다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

In [32]:
# 기본적인 경사하강법으로 최적화합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습도중 10번에 한번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.0993
20 1.09746
30 1.09565
40 1.09386
50 1.0921
60 1.09037
70 1.08866
80 1.08698
90 1.08532
100 1.08369


In [33]:
# 예측값인 model을 바로 출력하면 [0.2, 0.7, 0.1]과 같이 확률로 나오기 때문에,
# 요소 중 가장 큰 값의 인덱스를 찾아주는 argmax 함수를 사용하여 레이블 값을 출력하도록 했습니다.
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 1 0 0 0]
실제값: [0 1 2 0 0 2]


In [34]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 66.67
